# Calculate Weights for a Efficient Portfolio

In [414]:
import numpy as np
import pandas as pd

from scipy import stats
from scipy.optimize import minimize

import yfinance as yf

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool
from bokeh.models import ColorBar, LinearColorMapper, Range1d, Slope, Span
from bokeh.models import Label
from bokeh.transform import dodge
output_notebook()

Loading BokehJS ...

### Input Data

In [415]:
# stocks = ['GOOGL', 'AAPL', 'AMZN', 'MSFT', 'F', 'BMW.DE', 'TM', 'KO', 'PEP']
stocks        = ['CSPX.AS', 'IMAE.AS', 'EMIM.AS', 'CPXJ.AS', 'DBXG.DE', 'EPRA.PA']
range_weights = ((0.30, 1.0), (0., 0.7), (0., 0.7), (0., 0.7), (0., 0.7), (0., 0.7))
colors        = ['black', 'blue', 'green', 'red', 'fuchsia', 'turquoise']

# range of data
start, end = '2010-01-01', '2015-10-31'
# risk free rate return
risk_free_rate = 0.015
# number of trials
num_portfolios = 25000
# number of selected periods in a year
num_periods_year = 252.0

In [416]:
# data preparation
num_assets = len(tickers)

# download data
data = yf.download(stocks)
data = data.drop(columns=['Volume', 'Open', 'Low', 'High', 'Close'])['Adj Close']

# remove outliers from data
quantile_bound = 0.999999
for _ in range(2):  # do two passes for proper data cleaning
    for stock in list(data.columns):
        upper_bound = data[stock].quantile(quantile_bound)
        lower_bound = data[stock].quantile(1-quantile_bound)

        data[stock] = data[stock].mask(data[stock]>upper_bound, np.nan)
        data[stock] = data[stock].mask(data[stock]<lower_bound, np.nan)
        
# data.describe()

[*********************100%***********************]  6 of 6 completed


In [417]:
# plot prices
source = ColumnDataSource(data)

plot = figure(sizing_mode='stretch_both', x_axis_type='datetime')

for t, color in zip(stocks, colors):
    x = plot.line(x='Date', y=t, source=source, legend_label=t, line_color=color)
    plot.add_tools(HoverTool(mode='vline', tooltips=[('Date', '@Date{%F}'), (t, '@{'+f'{t}'+'}')], 
                             formatters={'@Date': 'datetime'},
                             line_policy='nearest', show_arrow=True, renderers=[x]))
    
plot.add_tools(CrosshairTool(dimensions='both', line_color='#808080'))

price_bound = max(data.max())*1.05
plot.y_range = Range1d(start=0, end=price_bound, bounds='auto')

plot.legend.location = 'top_left'
plot.legend.click_policy = 'hide'
plot.grid.visible = False

show(plot)

In [418]:
# calculate daily and annual returns of the stocks
returns_daily = np.log(data/data.shift(1))
returns_annual = returns_daily.mean()*num_periods_year

# get daily and covariance of returns of the stock
cov_daily = returns_daily.cov()
cov_annual = cov_daily*num_periods_year

In [431]:
# plot histogram of daily returns
hist_plots = []

for stock, color in zip(stocks, colors):
    hist, edges = np.histogram(returns_daily[stock], bins=25, 
                               range=(returns_daily[stock].min(), returns_daily[stock].max()),
                               density=True)

    plot = figure(sizing_mode='stretch_width', plot_height=250, plot_width=325)
    plot.quad(top=hist, left=edges[:-1], right=edges[1:], bottom=0, line_color="white", alpha=0.5, 
              legend_label=stock, fill_color=color)
    plot.add_tools(CrosshairTool(dimensions='both', line_color='#808080'))
    plot.y_range = Range1d(start=0, end=max(hist)*1.05, bounds='auto')
    plot.x_range = Range1d(start=-0.1, end=0.1, bounds='auto')
    
    plot.legend.location = 'top_left'
    plot.legend.click_policy = 'hide'
    plot.grid.visible = False
    
    hist_plots.append(plot)
    
show(gridplot(hist_plots, ncols=3))

In [420]:
#Run MC simulation of target trials portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []

#Calculate portfolios
count = 0
while count < num_portfolios:
    # get weights
    weights = []
    for r in range_weights:
        weights.append(np.random.uniform(low=r[0], high=r[1]))
    weights /= np.sum(weights)
    # check if all are valid
    checker = []
    for i, val in enumerate(range_weights):
        if weights[i] <= val[0] or weights[i] >= val[1]:
            checker.append(False)
        else:
            checker.append(True)
    
    if False in checker:
        pass
    else:
        # calculate expected return and volatility of portfolio
        pret, pvar = calc_portfolio_perf(weights, mean_returns_daily, cov_daily)

        # convert results to annual basis
        returns = pret*num_periods_year
        volatility = pvar*np.sqrt(num_periods_year)
        sharpe = (returns-risk_free_rate)/volatility

        # append
        port_returns.append(returns)
        port_volatility.append(volatility)
        sharpe_ratio.append(sharpe)
        count +=1
    
portfolios = pd.DataFrame({'Returns': port_returns,
                           'Volatility': port_volatility,
                           'Sharpe Ratio': sharpe_ratio})

# portfolios.describe()

In [421]:
#Find efficient frontier
target_returns = np.linspace(0.01, 0.30, 250)
efficient_portfolios = find_efficient_frontier(returns_annual, 
                                               cov_annual, 
                                               target_returns,
                                               range_weights)
efficient_var = [p['fun'] for p in efficient_portfolios]

#Find portfolio with maximum Sharpe ratio
max_sharpe = find_max_sharpe_ratio(returns_annual, cov_annual, 
                                   risk_free_rate, range_weights)
max_sharpe_r, max_sharpe_sd = calc_portfolio_perf(max_sharpe.x, 
                                                  returns_annual, 
                                                  cov_annual)

#Find portfolio with minimum variance
min_var = find_min_variance_portfolio(returns_annual, cov_annual, 
                                      range_weights)
min_var_r, min_var_sd = calc_portfolio_perf(min_var.x, returns_annual, 
                                            cov_annual)

In [422]:
# plot figure
source2 = ColumnDataSource(portfolios)
plot2 = figure(sizing_mode='stretch_both')


color_map = LinearColorMapper(palette='Turbo256', 
                              low=min(portfolios['Sharpe Ratio']), 
                              high=max(portfolios['Sharpe Ratio']))

plot2.circle(x='Volatility', y='Returns', source=source2, 
             fill_color={'field': 'Sharpe Ratio', 'transform': color_map}, line_color=None)

# min volatility
plot2.circle(min_var_sd, min_var_r, size=15, fill_color='blue', line_color=None, 
             legend_label='Minimum Volatility')
# max sharpe
plot2.circle(max_sharpe_sd, max_sharpe_r, size=15, fill_color='red', line_color=None, 
             legend_label='Maximum Sharpe')

# plot efficient frontier
plot2.line(efficient_var, target_returns, line_color='gray', line_width=2, 
           legend_label='Efficient Frontier')

# build Capital Allocation Line
cal_m, cal_b, _, _, _ = stats.linregress([0, float(max_sharpe_sd)], 
                                         [risk_free_rate, float(max_sharpe_r)])
plot2.add_layout(Slope(gradient=cal_m, y_intercept=cal_b))
plot2.add_layout(Label(x=0.001, y=risk_free_rate, x_units='data', y_units='data', 
                       text_baseline='top', text='Capital Market Line'))

# add min volatility span
plot2.add_layout(Span(location=float(min_var_r), dimension='width'))
plot2.add_layout(Label(x=0.001, y=float(min_var_r), x_units='data', y_units='data', 
                       text='Efficency Bound'))

plot2.add_tools(CrosshairTool(dimensions='both', line_color='#808080'))

# color bar
color_bar = ColorBar(color_mapper=color_map, location=(0,0))
plot2.add_layout(color_bar, 'right')

plot2.x_range = Range1d(start=0, 
                        end=portfolios['Volatility'].max()*1.1, 
                        bounds='auto')
plot2.y_range = Range1d(start=0, 
                        end=portfolios['Returns'].max()*1.1, 
                        bounds='auto')

plot2.legend.location = 'top_left'
plot2.legend.click_policy = 'hide'
plot2.grid.visible = False

show(plot2)

In [423]:
# plot weigths of best portfolios
targets = pd.DataFrame({'Min Volatility': min_var.x, 'Max Sharpe': max_sharpe.x},
                        index=stocks)
source3 = ColumnDataSource(targets)

plot3 = figure(x_range=stocks, sizing_mode='stretch_both')

v = plot3.vbar(x=dodge('index', -0.2, range=plot3.x_range), top='Min Volatility', source=source3, width=0.4, 
               legend_label='Min volatility', color='blue')

s = plot3.vbar(x=dodge('index', 0.2, range=plot3.x_range), top='Max Sharpe', source=source3, width=0.4, 
               legend_label='Max Sharpe', color='red')

plot3.add_tools(HoverTool(mode='vline', 
                          tooltips=[('Weight', '@{Min Volatility}{0.00%}')],
                          renderers=[v]))
plot3.add_tools(HoverTool(mode='vline', 
                          tooltips=[('Weight', '@{Max Sharpe}{0.00%}')],
                          renderers=[s]))

plot3.y_range = Range1d(start=0, end=1, bounds='auto')
plot3.xgrid.visible = False


text1 = f'Min volatility - return: {round(float(min_var_r), 2)}, volatility: {round(float(min_var_sd), 2)}'
plot3.add_layout(Label(x=0.01, y=0.9, x_units='data', y_units='data', 
                       text=text1))

text2 = f'Best risk-adjusted - return: {round(float(max_sharpe_r), 2)} volatility: {round(float(max_sharpe_sd), 2)}'
plot3.add_layout(Label(x=0.01, y=0.85, x_units='data', y_units='data', 
                       text=text2))

show(plot3)

In [424]:
## functions needed to run the script
def calc_portfolio_perf(weights, mean_returns, cov_matrix):
    ''' Calculates the expected mean of returns and volatility for a 
    portolio of assets, each carrying the weight specified by weights
    
    INPUT
    weights: array specifying the weight of each asset in the portfolio
    mean_returns: mean values of each asset's returns
    cov_matrix: covariance of each asset in the portfolio
    
    OUTPUT
    tuple containing the portfolio return and volatility
    '''    

    p_ret = np.sum(np.dot(mean_returns, weights))
    p_var = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    return p_ret, p_var


def get_portfolio_vol(weights, mean_returns, cov_matrix):
    ''' Returns the volatility of the specified portfolio of assets '''
    
    return calc_portfolio_perf(weights, mean_returns, cov_matrix)[1]


def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    ''' Returns the negated Sharpe Ratio for the speicified portfolio of assets
    
    INPUT
    weights: array specifying the weight of each asset in the portfolio
    mean_returns: mean values of each asset's returns
    cov_matrix: covariance of each asset in the portfolio
    risk_free_rate: time value of money
    '''
    p_ret, p_var = calc_portfolio_perf(weights, mean_returns, cov_matrix)
    
    return -(p_ret-risk_free_rate)/p_var


def _data_preparation(mean_returns, cov_matrix, risk_free_rate):
    ''' Prepare comon stuff '''
    
    num_assets = len(mean_returns)
    x0 =  num_assets*[1./num_assets,]
    
    args = (mean_returns, cov_matrix, risk_free_rate)
    if risk_free_rate is None:
        args = (mean_returns, cov_matrix)
        
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    return args, x0, constraints


def find_max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate, bounds):
    '''Finds the portfolio of assets providing the maximum Sharpe Ratio
    
    INPUT
    mean_returns: mean values of each asset's returns
    cov_matrix: covariance of each asset in the portfolio
    risk_free_rate: time value of money
    '''
    
    args, x0, constraints = _data_preparation(mean_returns, cov_matrix, 
                                              risk_free_rate)
    
    return minimize(neg_sharpe_ratio, x0, args=args, method='SLSQP', 
                    bounds=bounds, constraints=constraints)


def find_min_variance_portfolio(mean_returns, cov_matrix, bounds):
    '''Finds the portfolio of assets providing the lowest volatility
    
    INPUT
    mean_returns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    '''
    
    args, x0, constraints = _data_preparation(mean_returns, cov_matrix, 
                                              None)
    
    return minimize(get_portfolio_vol, x0, args=args, method='SLSQP', 
                    bounds=bounds, constraints=constraints)


def find_efficient_return(mean_returns, cov_matrix, target_return, bounds):
    ''' Finds the portfolio of assets providing the target return with lowest
    volatility
    
    INPUT
    mean_returns: mean values of each asset's returns
    cov_matrix: covariance of each asset in the portfolio
    target_return: APR of target expected return
    
    OUTPUT
    Dictionary of results from optimization
    '''
    
    def get_portfolio_return(weights):
        return calc_portfolio_perf(weights, mean_returns, cov_matrix)[0]
    
    constraints = ({'type': 'eq', 'fun': lambda x: get_portfolio_return(x)-target_return},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    args, x0, _ = _data_preparation(mean_returns, cov_matrix, None)
    
    return minimize(get_portfolio_vol, x0, args=args, method='SLSQP', 
                    bounds=bounds, constraints=constraints)


def find_efficient_frontier(mean_returns, cov_matrix, target_returns, bounds):
    ''' Finds the set of portfolios comprising the efficient frontier
    
    INPUT
    mean_returns: mean values of each asset's returns
    cov_matrix: covariance of each asset in the portfolio
    target_returns: APR of target expected return
    
    OUTPUT
    Dictionary of results from optimization
    '''
    
    efficient_portfolios = []
    for ret in target_returns:
        result = find_efficient_return(mean_returns, cov_matrix, ret, bounds)
        efficient_portfolios.append(result)
        
    return efficient_portfolios